#### For Colab

In [2]:
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click() 
}
var i = setInterval(ClickConnect, 900000)
clearInterval(i)
"""

'\nfunction ClickConnect(){\n    console.log("Working");\n    document.querySelector("colab-toolbar-button").click() \n}\nvar i = setInterval(ClickConnect, 900000)\nclearInterval(i)\n'

In [3]:
!nvidia-smi

Sat Mar 20 11:54:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
drive_dir = '/content/drive/My Drive/MAGMA: Summarization/'

#### Install Libraries

In [6]:
!cd '/content/drive/My Drive/MAGMA: Summarization/transformers/'; pip install -q -e .
!pip install -q -U sentence-transformers
!pip install -q -U wandb
!pip install -q -U rouge-score sacrebleu

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2MB 7.5MB/s 
     |████████████████████████████████| 890kB 51.4MB/s 
     |████████████████████████████████| 81kB 6.2MB/s 
     |████████████████████████████████| 1.2MB 13.8MB/s 
     |████████████████████████████████| 2.0MB 8.0MB/s 
     |████████████████████████████████| 133kB 49.0MB/s 
     |████████████████████████████████| 163kB 56.9MB/s 
     |████████████████████████████████| 102kB 14.2MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 


In [7]:
print("Setting up colab environment")
!pip uninstall -y -q pyarrow
!pip install -q -U ray[tune]
!pip install -q ray[debug]

Setting up colab environment
     |████████████████████████████████| 47.5MB 65kB/s 
     |████████████████████████████████| 133kB 58.1MB/s 
     |████████████████████████████████| 1.3MB 53.5MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
     |████████████████████████████████| 81kB 12.9MB/s 
     |████████████████████████████████| 3.2MB 46.3MB/s 
     |████████████████████████████████| 204kB 59.3MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
     |████████████████████████████████| 317kB 57.5MB/s 
     |████████████████████████████████| 296kB 59.0MB/s 
     |████████████████████████████████| 143kB 59.1MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 


In [8]:
import sys

print("\n##### sys.path original #####")
for p in sys.path:
    print(f"'{p}'")
print()

import site
site.main()

print("\n##### sys.path after site.main() #####")
for p in sys.path:
    print(f"'{p}'")
print()


##### sys.path original #####
''
'/content'
'/env/python'
'/usr/lib/python37.zip'
'/usr/lib/python3.7'
'/usr/lib/python3.7/lib-dynload'
'/usr/local/lib/python3.7/dist-packages'
'/usr/lib/python3/dist-packages'
'/usr/local/lib/python3.7/dist-packages/IPython/extensions'
'/root/.ipython'


##### sys.path after site.main() #####
'/content'
'/env/python'
'/usr/lib/python37.zip'
'/usr/lib/python3.7'
'/usr/lib/python3.7/lib-dynload'
'/usr/local/lib/python3.7/dist-packages'
'/usr/lib/python3/dist-packages'
'/usr/local/lib/python3.7/dist-packages/IPython/extensions'
'/root/.ipython'
'/content/drive/My Drive/MAGMA: Summarization/transformers/src'



### **Config**

In [9]:
import sys
sys.path.insert(0, drive_dir)
import config

import wandb
wandb.login()

project_name = 'ft_pegasus_bull_para_embed_bybook'
%env WANDB_PROJECT=$project_name

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=ft_pegasus_para_wordembed_bybook


In [10]:
finetune_script = '"'+drive_dir+'transformers/examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval.py"'
gps_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval_search.py"'

In [11]:
model_name_or_path = 'google/pegasus-large'

In [12]:
data_dir = '"'+drive_dir+'datasets/karger_books_para_wordembed/pegasus/st/base/bybook/"'

source_val_dir = data_dir[:-1] + 'val.source"'
reference_val_dir = data_dir[:-1] + 'val.target"'

source_test_dir = data_dir[:-1] + 'test.source"'
reference_test_dir = data_dir[:-1] + 'test.target"'

In [13]:
GAS = 64
LR = 5e-5

output_dir = '"'+drive_dir+'fine-tuning/'+project_name+'_gas'+\
    str(GAS)+'_lr' + str(LR) + '"'

### Evaluation Out of the Box

#### Validation

In [14]:
eval_model_name_or_path = '"google/pegasus-large"'

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_bybook_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_bybook_val.json"'
print(save_dir)

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_bybook_gas64_lr5e-05/google?pegasus-large_para_wordembed_val.txt"
100% 306M/306M [00:18<00:00, 16.4MB/s]
parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
Downloading: 100% 2.87k/2.87k [00:00<00:00, 1.69MB/s]
Downloading: 100% 2.28G/2.28G [01:01<00:00, 37.2MB/s]
Downloading: 100% 1.91M/1.91M [00:00<00:00, 4.47MB/s]
Downloading: 100% 65.0/65.0 [00:00<00:00, 96.1kB/s]
Downloading: 100% 88.0/88.0 [00:00<00:00, 112kB/s]
  0% 0/25 [00:00<?, ?it/s]2021-03-20 12:01:42.620157: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 25/25 [03:25<00:00,  8.23s/it]
{'rouge1_precision': 24.59, 'rouge1_recall': 31.53, 'rouge1_fmeasure': 25.91, 'rouge2_precision': 9.6, 'rouge2_recall': 11.21, 'rouge2_fmeasure': 9.59, 'rougeL_precision': 19.55, 'rougeL_recall': 24.89, 'rougeL_fmeasure': 20.53, 'rougeLsum_precision': 20.11, 'ro

#### Test

In [15]:
eval_model_name_or_path = '"google/pegasus-large"'

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_bybook_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_bybook_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/34 [00:00<?, ?it/s]2021-03-20 12:06:50.247195: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 34/34 [05:12<00:00,  9.18s/it]
{'rouge1_precision': 24.990000000000002, 'rouge1_recall': 34.37, 'rouge1_fmeasure': 27.250000000000004, 'rouge2_precision': 8.35, 'rouge2_recall': 11.76, 'rouge2_fmeasure': 9.07, 'rougeL_precision': 18.740000000000002, 'rougeL_recall': 26.150000000000002, 'rougeL_fmeasure': 20.51, 'rougeLsum_precision': 19.49, 'rougeLsum_recall': 27.0, 'rougeLsum_fmeasure': 21.22, 'sentence_distilroberta_cosine': 55.806589126586914, 'w2v_cosine': 99.99589323997498, 'n_obs': 272, 'runtime': 312, 'seconds_per_sample': 1.1471, 'min_length': 10, 'max_length': 150}


### Fine-tuning

In [ ]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.num_beams = 2

model_config_dir = '"'+drive_dir+'fine-tuning/config/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

In [ ]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--data_dir $data_dir \
--freeze_embeds \
--adafactor \
--do_train \
--learning_rate $LR \
--label_smoothing_factor 0.1 \
--warmup_steps 0 \
--sortish_sampler \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--max_steps 700 \
--logging_steps 7 --logging_first_step \
--per_device_train_batch_size 2 --per_device_eval_batch_size 8 \
--gradient_accumulation_steps $GAS \
--evaluation_strategy steps --eval_steps 7 --eval_beams 2 \
--predict_with_generate \
--save_steps 49 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

Streaming output truncated to the last 5000 lines.
100%|██████████| 25/25 [00:54<00:00,  2.16s/it]

Batches: 100%|██████████| 1/1 [00:00<00:00, 99.66it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 106.17it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 110.87it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 109.30it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 108.25it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 106.88it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 111.81it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 102.07it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 112.91it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 109.78it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 112.50it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 111.43it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 113.26it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 106.60it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 109.17it/s]


Batches: 100%|██████████| 1/1 [

### Evaluation Before GPS

In [16]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-686"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_bybook_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_bybook_val.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_bybook_gas64_lr5e-05/checkpoint-686"

parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/25 [00:00<?, ?it/s]2021-03-20 12:13:26.449723: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 25/25 [00:47<00:00,  1.91s/it]
{'rouge1_precision': 34.2, 'rouge1_recall': 33.879999999999995, 'rouge1_fmeasure': 32.05, 'rouge2_precision': 14.67, 'rouge2_recall': 14.299999999999999, 'rouge2_fmeasure': 13.669999999999998, 'rougeL_precision': 27.82, 'rougeL_recall': 27.36, 'rougeL_fmeasure': 25.94, 'rougeLsum_precision': 28.67, 'rougeLsum_recall': 28.16, 'rougeLsum_fmeasure': 26.790000000000003, 'sentence_distilroberta_cosine': 60.669541358947754, 'w2v_cosine': 99.99569654464722, 'n_obs': 195, 'runtime': 47, 'seconds_per_sample': 0.241, 'min_length': 10, 'max_length': 150}


In [17]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-686"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_bybook_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_bybook_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_bybook_gas64_lr5e-05/checkpoint-686"

parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/34 [00:00<?, ?it/s]2021-03-20 12:16:02.909203: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 34/34 [01:01<00:00,  1.80s/it]
{'rouge1_precision': 33.58, 'rouge1_recall': 32.36, 'rouge1_fmeasure': 31.080000000000002, 'rouge2_precision': 13.83, 'rouge2_recall': 12.82, 'rouge2_fmeasure': 12.520000000000001, 'rougeL_precision': 26.58, 'rougeL_recall': 25.52, 'rougeL_fmeasure': 24.54, 'rougeLsum_precision': 27.54, 'rougeLsum_recall': 26.369999999999997, 'rougeLsum_fmeasure': 25.34, 'sentence_distilroberta_cosine': 58.60275626182556, 'w2v_cosine': 99.99597072601318, 'n_obs': 272, 'runtime': 61, 'seconds_per_sample': 0.2243, 'min_length': 10, 'max_length': 150}


### GPS

In [18]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-686"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_gps.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_gps.json"'

!python3 $gps_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--task summarization \
--search="length_penalty=0.1:0.8:1:1.9 num_beams=2:4:6 no_repeat_ngram_size=3:4:5" \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_bybook_gas64_lr5e-05/checkpoint-686"

  0% 0/25 [00:00<?, ?it/s]2021-03-20 12:18:55.860031: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 25/25 [00:43<00:00,  1.76s/it]
100% 25/25 [00:40<00:00,  1.62s/it]
100% 25/25 [00:41<00:00,  1.67s/it]
100% 25/25 [01:04<00:00,  2.57s/it]
100% 25/25 [01:01<00:00,  2.44s/it]
100% 25/25 [01:01<00:00,  2.44s/it]
100% 25/25 [01:26<00:00,  3.46s/it]
100% 25/25 [01:27<00:00,  3.49s/it]
100% 25/25 [01:29<00:00,  3.58s/it]
100% 25/25 [00:43<00:00,  1.73s/it]
100% 25/25 [00:44<00:00,  1.76s/it]
100% 25/25 [00:50<00:00,  2.00s/it]
100% 25/25 [01:08<00:00,  2.73s/it]
100% 25/25 [01:09<00:00,  2.79s/it]
100% 25/25 [01:10<00:00,  2.81s/it]
100% 25/25 [01:48<00:00,  4.34s/it]
100% 25/25 [01:51<00:00,  4.46s/it]
100% 25/25 [01:58<00:00,  4.74s/it]
100% 25/25 [00:44<00:00,  1.79s/it]
100% 25/25 [00:44<00:0

### Evaluation

In [19]:
### Parameters found through Generative Parameter Search and visual representation
### check the file gps_pegasus_para_wordembed_merge

min_length = config.ONE_BULLET_MIN_LEN
length_penalty = 1
num_beams = 2
no_repeat_ngram_size = 4

In [20]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-686"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_bybook_gps_val.txt"'
score_dir = output_dir[:-1] + eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_bybook_gps_val.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $min_length \
--length_penalty $length_penalty \
--num_beams $num_beams \
--no_repeat_ngram_size $no_repeat_ngram_size \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_bybook_gas64_lr5e-05/checkpoint-686"

parsed the following generate kwargs: {'min_length': 10, 'length_penalty': 1, 'num_beams': 2, 'no_repeat_ngram_size': 4, 'max_length': 150}
  0% 0/25 [00:00<?, ?it/s]2021-03-20 13:54:54.393394: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 25/25 [00:49<00:00,  1.99s/it]
{'rouge1_precision': 34.11, 'rouge1_recall': 34.949999999999996, 'rouge1_fmeasure': 32.5, 'rouge2_precision': 15.260000000000002, 'rouge2_recall': 14.719999999999999, 'rouge2_fmeasure': 14.13, 'rougeL_precision': 27.6, 'rougeL_recall': 27.900000000000002, 'rougeL_fmeasure': 26.06, 'rougeLsum_precision': 28.49, 'rougeLsum_recall': 28.82, 'rougeLsum_fmeasure': 26.88, 'sentence_distilroberta_cosine': 61.25394105911255, 'w2v_cosine': 99.99563097953796, 'n_obs': 195, 'runtime': 49, 'seconds_per_sample': 0.2513, 'min_length': 10, 

In [21]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-686"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_bybook_gps_test.txt"'
score_dir = output_dir[:-1] + eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_bybook_gps_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $min_length \
--length_penalty $length_penalty \
--num_beams $num_beams \
--no_repeat_ngram_size $no_repeat_ngram_size \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_bybook_gas64_lr5e-05/checkpoint-686"

parsed the following generate kwargs: {'min_length': 10, 'length_penalty': 1, 'num_beams': 2, 'no_repeat_ngram_size': 4, 'max_length': 150}
  0% 0/34 [00:00<?, ?it/s]2021-03-20 13:57:50.640130: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 34/34 [01:05<00:00,  1.94s/it]
{'rouge1_precision': 33.660000000000004, 'rouge1_recall': 32.45, 'rouge1_fmeasure': 31.14, 'rouge2_precision': 14.04, 'rouge2_recall': 12.889999999999999, 'rouge2_fmeasure': 12.65, 'rougeL_precision': 26.68, 'rougeL_recall': 25.56, 'rougeL_fmeasure': 24.59, 'rougeLsum_precision': 27.529999999999998, 'rougeLsum_recall': 26.3, 'rougeLsum_fmeasure': 25.3, 'sentence_distilroberta_cosine': 58.788299560546875, 'w2v_cosine': 99.9962329864502, 'n_obs': 272, 'runtime': 65, 'seconds_per_sample': 0.239, 'min_length': 10, 'length_penalt